### Autograd
* 通过设置.requires_grad=True 来告诉Tensor类自动求导，会记住每一步操作历史.也可以冻结模型
* 最后调用.backward()自动计算梯度，相对于某个标量值，保存到grad属性中, x.grad也是一个张量
* grad_fn记录y是怎么来的，即y的公式

In [6]:
import torch, torchvision

x = torch.ones(2, 2, requires_grad=True)
print(x)
y = torch.zeros(2, 2, requires_grad=True)
print(y)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[0., 0.],
        [0., 0.]], requires_grad=True)


In [7]:
z = torch.sum(x + y)
print(z)

tensor(4., grad_fn=<SumBackward0>)


In [8]:
# 自动求导， 对各个变量求导
z.backward()
print(x.grad,"\n", y.grad)

tensor([[1., 1.],
        [1., 1.]]) 
 tensor([[1., 1.],
        [1., 1.]])


In [23]:
x = torch.IntTensor([1, 2, 3])
y = torch.IntTensor([1, 1, 1])
h = x + y
print(h, h.dtype)
print(h.grad)
print(h.grad_fn)

# h.backward()    # x,y没有梯度记录
print(x.grad)

tensor([2, 3, 4], dtype=torch.int32) torch.int32
None
None
None


In [32]:
dtype = torch.float
device = torch.device("cpu")

a = torch.randn([ 2, 3], device=device, dtype=dtype, requires_grad=True)
b = torch.randn([ 2, 3], device=device, dtype=dtype, requires_grad=True)

h = torch.sum(a + b)
print(a)
print(h, h.dtype)
print("h.grad", h.grad)
h.backward()
print("h.grad", h.grad)    # h没有梯度，因为没有对他求导，是对参数求导
print(x.grad)

tensor([[ 0.5583, -1.5237, -0.2370],
        [ 0.7126,  0.9256,  0.3143]], requires_grad=True)
tensor(2.0965, grad_fn=<SumBackward0>) torch.float32
h.grad None
h.grad None
None


<ipython-input-32-17f491f8072d>:10: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  print("h.grad", h.grad)
<ipython-input-32-17f491f8072d>:12: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  print("h.grad", h.grad)


In [32]:
# 梯度置零
x.grad.zero_()
print(x.grad)

tensor([[3., 3.],
        [3., 3.]]) 
 tensor([[6., 6.],
        [6., 6.]])
tensor([[0., 0.],
        [0., 0.]])


In [31]:
h = x**2 + y**3
print(h)
h.backward(torch.ones_like(x))
print(x.grad)

tensor([[2., 2.],
        [2., 2.]], grad_fn=<AddBackward0>)
tensor([[2., 2.],
        [2., 2.]])


### 示例

In [2]:
import torch, torchvision

# 加载预训练模型
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

# 正向传播
prediction = model(data)

# 误差张量
loss = (prediction - labels).sum()
# 误差张量反向传播，autograd为每个模型参数计算梯度并存储在参数的.grad属性中
loss.backward()

# 加载SGD优化器，在优化器中注册模型的所有参数
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
# 调用.step()启动梯度下降，优化器通过.grad中的梯度来调整每个参数
optim.step()


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/wukong/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
# DAG中排除，冻结参数，已调整预训练网络

from torch import nn, optim

model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    
# 将分类器最后一个线性层解冻
model.fc = nn.Linear(512, 10)

optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)

#### with torch.no_grad()

In [4]:
# with torch.no_grad() ： 清除w的梯度
# 在该模块下，所有计算得出的tensor的requires_grad都自动设置为False，即使本来为True
# with torch.no_grad()则主要是用于停止autograd模块的工作，以起到加速和节省显存的作用。
# 它的作用是将该with语句包裹起来的部分停止梯度的更新，不会影响dropout层和batchnorm层
# 从而节省了GPU算力和显存，但是并不会影响dropout和BN层的行为。
x = torch.randn(10, 5, requires_grad = True)
y = torch.randn(10, 5, requires_grad = True)
z = torch.randn(10, 5, requires_grad = True)
with torch.no_grad():
    w = x + y + z
    print(w.requires_grad)
    print(w.grad_fn)
print(w.requires_grad)
print(x.requires_grad)
print(x.grad_fn)        # x是叶子节点,不存在grad_fn和grad

False
None
False
True
None


#### with torch.set_grad_enabled()
* 与with troch.no_grad 相似，会将在这个with包裹下的所有的计算出的 新的变量 的required_grad 置为false。但原有的变量required_grad 不会改变。这实际上也就是影响了网络的自动求导机制。与with torch.no_grad() 相似，不过接受一个bool类型的值。

In [ ]:
# 设置为False才有效
torch.set_grad_enabled(False)